In [20]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout,BatchNormalization
from keras.constraints import maxnorm
from keras.optimizers import SGD, Adam
import keras.utils
import sys
import tensorflow as tf
import datetime
from keras.callbacks import ModelCheckpoint

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [21]:
def get_validation_np():
    dataset = np.load('../data/npy/validation.npy',mmap_mode='r')
    X = dataset[:,0:92]
    Y = dataset[:,92:] # :96
    return X, Y

In [22]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, batch_size=1024, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data = np.load('../data/npy/train.npy',mmap_mode='r')
        # self.data = np.memmap('full_dataset/train.buffer', dtype=np.float16, mode='r',shape=(255519715, 99))
        # rows = int(self.data.shape[0] / 99)
        # self.data = np.reshape(self.data,(rows,99))
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(self.data.shape[0] / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.data.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples'
        X = np.empty((self.batch_size, 92),dtype=np.float16)
        y = np.empty((self.batch_size, 7),dtype=np.float16)
        for i, ID in enumerate(list_IDs_temp):
            X[i,] = self.data[ID,:92]
            y[i] = self.data[ID,92:]
        return X, y

In [37]:
def split_activation(l):
    l_s = tf.keras.activations.sigmoid(l[...,0:3])
    l_t = tf.keras.activations.tanh(l[...,3:])
    lnew = tf.concat([l_s, l_t], axis = 1)
    return lnew

def get_model():
    model = Sequential()
    
    hidden = 2
    widest_layer = 256
    output_width = 7
    
    # model.add(Dense(92, input_shape = (92,)))
    # model.add(Dropout(0.1))
    model.add(BatchNormalization(input_shape = (92,)))
    for i in range(hidden):
        size = widest_layer# - i*widest_layer/hidden
        # model.add(Dense(size,kernel_constraint=maxnorm()))
        model.add(Dense(size,use_bias=False))#,kernel_constraint=maxnorm()))
        model.add(BatchNormalization())
        # model.add(Activation("relu"))
        model.add(tf.keras.layers.LeakyReLU())
    
    model.add(Dense(output_width,activation=split_activation))#,kernel_constraint=maxnorm()))
    sgd = SGD(lr=0.01, momentum=0.99, nesterov=False)
    weights = calculating_class_weights()
    # weights = np.array([[0.59429336, 3.15130026], [ 0.51234971, 20.74339526], [0.51744448, 14.83118107 ]])
    model.compile(loss=get_weighted_loss(weights), optimizer='adam', metrics=[])
    
    return model

In [36]:
def calculating_class_weights():
    y = np.load('../data/npy/train.npy',mmap_mode='r')
    # y = np.memmap('full_dataset/train.buffer', dtype=np.float16, mode='r',shape=(255519715, 99))
    # rows = int(y.shape[0] / 99)
    # y = np.reshape(y,(rows,99))
    y = y[:,92:95]
    y_labels = np.array(y)
    y_labels = np.round(y_labels)
    del y
    from sklearn.utils.class_weight import compute_class_weight
    number_dim = np.shape(y_labels)[1]
    weights = np.empty([number_dim, 2])
    for i in range(number_dim):
        weights[i] = compute_class_weight('balanced', [0.,1.], y_labels[:, i])
        print(weights[i])
    # weights[1] = weights[1]*1.5 # trying to put more emphasis on correctness of jumps
    # weights[3] = weights[3]*1.5 # and dodges
    return weights

def get_weighted_loss(weights):
    def weighted_loss(y_true, y_pred):
        # y_true = tf.where(tf.math.is_nan(y_true), y_pred, y_true) # assume correct if nan in true values [meaning that guess does not matter]
        y_true_binary = y_true[:,0:3]
        y_pred_binary = y_pred[:,0:3]
        y_true_analog = y_true[:,3:]
        y_pred_analog = y_pred[:,3:]
        loss_binary = keras.backend.mean((weights[:,0]**(1-y_true_binary))*(weights[:,1]**(y_true_binary))*keras.backend.binary_crossentropy(y_true_binary, y_pred_binary),axis=0)
        loss_analog = keras.backend.abs(keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_true_analog-y_pred_analog)),axis=0)))
        # squared_error = keras.backend.square((y_true_analog-y_pred_analog))
        # squared_weighted_error = squared_error*[1.0,0.0,1.0,3.0]
        # loss_analog = keras.backend.abs(keras.backend.sqrt(keras.backend.mean(squared_weighted_error,axis=-1)))
        y = tf.concat((loss_binary,loss_analog),axis=0)
        # y = loss_binary*3./7. + loss_analog*4./7.
        y = y*[1.0,1.0,1.0,1.0,1.0,1.0,2.0] # steer/yaw importance * 2
        # return keras.backend.mean(y)
        return y
    return weighted_loss

In [39]:
model = get_model()
# weights = np.array([[0.59429336, 3.15130026], [ 0.51234971, 20.74339526], [0.51744448, 14.83118107 ]])
# model = keras.models.load_model('../models/latest.h5',custom_objects={'split_activation': split_activation, 'weighted_loss': get_weighted_loss(weights)})
# log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
mc = ModelCheckpoint('../models/best_{epoch:02d}_{val_loss:.4f}.h5', monitor='val_loss', save_best_only=True, verbose=1)
mc2 = ModelCheckpoint('../models/latest.h5', monitor='val_loss', save_best_only=False, verbose=1)
X_val, Y_val = get_validation_np()
g = DataGenerator(batch_size=128, shuffle=True)
model.fit(g, validation_data=(X_val,Y_val), epochs=5, callbacks=[mc,mc2], verbose=1, workers=4, max_queue_size=32, initial_epoch=0)

[0.59566412 3.11330993]
[ 0.51033345 24.69328   ]
[ 0.51292848 19.83714653]
Epoch 1/10
1204/1205 [============================>.] - ETA: 0s - loss: 0.4850
Epoch 00001: val_loss improved from inf to 0.53076, saving model to best_01_0.5308.h5

Epoch 00001: saving model to latest.h5
1205/1205 [==============================] - 8s 6ms/step - loss: 0.4849 - val_loss: 0.5308
Epoch 2/10
1199/1205 [============================>.] - ETA: 0s - loss: 0.4368
Epoch 00002: val_loss did not improve from 0.53076

Epoch 00002: saving model to latest.h5
1205/1205 [==============================] - 7s 6ms/step - loss: 0.4369 - val_loss: 0.5360
Epoch 3/10
1198/1205 [============================>.] - ETA: 0s - loss: 0.4183
Epoch 00003: val_loss did not improve from 0.53076

Epoch 00003: saving model to latest.h5
1205/1205 [==============================] - 7s 6ms/step - loss: 0.4183 - val_loss: 0.5598
Epoch 4/10
1193/1205 [============================>.] - ETA: 0s - loss: 0.4043
Epoch 00004: val_loss did n